In [ ]:
pip install -e ../../.

In [ ]:
import sunpy.map

import matplotlib.pyplot as plt

from sdoml import SDOMLDataset
from timeit import default_timer as timer

First, we will instantiate the ``SDOMLDataset`` class, to load two years of 
the three components of Magnetogram data from SDO/HMI channels (Bx,By,Bz) 
from ``fdl-sdoml-v2/sdomlv2_hmi.zarr``

In [ ]:
sdomlds = SDOMLDataset(
    cache_max_size=1 * 512 * 512 * 4096,
    years=["2010"],
    data_to_load={
        "HMI": {
            "location": "gcs",
            "root": "fdl-sdoml-v2/sdomlv2_hmi_small.zarr/",
            "channels": ["Bx", "By", "Bz"],
        }, 
    },
)

With the Dataset instantiated, we will directly access the dataset using the ``__getitem__`` method. ``sdomlds.__getitem__(idx)`` loads and returns a sample from the dataset at the given index ``idx``.  

As will be evident, the first data access for this index is relatively slow (it is retrieved from remote store on Google Cloud Storage), however the second data access is faster, as this uses cache. For more information see https://zarr.readthedocs.io/en/stable/api/storage.html#zarr.storage.LRUStoreCache

In [ ]:
# obtain the first item of the dataset
idx = 0

# extract image and metadata:
images, metadata = sdomlds.__getitem__(idx)

``SDOMLDataset()`` returns both image, and metadata. 

For the example here, the full image dataset is of size ``(1138, 3, 512, 512)``, where there are 6133 co-temporal observations, each with the 3 components of the magnetic field vector ``(512, 512)``.

* The ``images`` returned by ``__getitem__(idx)`` are of size: ``(1, 3, 512, 512)``, where each of the 1138 items consist of the 3 components of the SDO/HMI magnetograms (``[Bx, By, Bz]``) of ``torch.Size([512, 512])``. 


* The ``metadata`` is a dictionary of 147 key-value pairs. Each keys value is a list of 3 elements (which directly corresponds to the 3 channels in the image data). A small excerpt of this dictionary is shown below for ``['WAVELNTH', 'T_OBS']``:


```
{
    'WAVELNTH': [6173.0, 6173.0, 6173.0],
    'T_OBS': 
        ['2010.05.01_10:00:04_TAI',
         '2010.05.01_10:00:04_TAI',
         '2010.05.01_10:00:04_TAI'],
}
```

### Plotting one set of images

For the one set of images returned from the dataloader, the following code block creates the set of ``sunpy.map`` from the ``images`` and ``metadata``.

In [ ]:
images = images[0]
metadata = metadata[0]

In [ ]:
plt.figure(figsize=(20, 12))

for img_index in range(images[0,:,0,0].shape[0]):
    # Create a sunpy map with the data
    selected_image = images[0,img_index,:,:]
    selected_headr = {keys: values[img_index] for keys, values in metadata.items()}
    my_map = sunpy.map.Map(selected_image.numpy(), selected_headr)
    
    # set the index and plot the sunpy.map
    ax = plt.subplot(1, 3, img_index+1, projection=my_map)
    
    my_map.plot_settings['cmap'] = "hmimag"
    my_map.plot_settings['norm'] = plt.Normalize(-1000.,1000.)

    my_map.plot()